In [ ]:
import torch
import numpy as np
import os
os.chdir('..')

import torch_converter as tc
import instance_generator as ig
from torch_geometric.loader import DataLoader
from gnn_library.util import train, save, load
from util import NumpyDataset, collect
from evaluate import evaluate_model, evaluate_meta_model, pp_output

%load_ext autoreload
%autoreload 2

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("PyTorch has version {}".format(torch.__version__))
print('Using device:', device)

In [ ]:
GNN1, args1 = load('GNN1')
GNN2, args2 = load('GNN2')

In [ ]:
train_num = 100; test_num = 30

er_config = {
    'graph_type': 'ER',
    'p': 1,
    'weighted': True
}
ba_config = {
    'graph_type': 'BA',
    'ba_param': 2,
    'weighted': True
}
geom_config = {
    'graph_type': 'GEOM',
    'threshold': 0.2,
    'scaling': 1 / np.sqrt(2)
}

rng = np.random.default_rng()

train_instances, train_coin_flips = collect(
    [
        ig.sample_instances(10, 6, train_num, rng, **er_config),
        ig.sample_instances(10, 6, train_num, rng, **ba_config),
        ig.sample_instances(10, 6, train_num, rng, **geom_config),
    ]
)

test_instances, test_coin_flips = collect(
    [
        ig.sample_instances(10, 6, test_num, rng, **er_config),
        ig.sample_instances(10, 6, test_num, rng, **ba_config),
        ig.sample_instances(10, 6, test_num, rng, **geom_config),
    ]
)

train_dataset = NumpyDataset(
    tc._instances_to_nn_samples(
        instances=train_instances,
        coin_flips=train_coin_flips,
        models=[GNN1, GNN2],
        args=args1,
        batch_size=50,
        rng=rng
    )
)

test_dataset = NumpyDataset(
    tc._instances_to_nn_samples(
        instances=test_instances,
        coin_flips=test_coin_flips,
        models=[GNN1, GNN2],
        args=args1,
        batch_size=50,
        rng=rng
    )
)

args = {
    'processor':         'NN',
    'head':              'meta',
    'num_layers':        5,
    'input_dim':         17,
    'batch_size':        4,
    'hidden_dim':        16,
    'output_dim':        3,
    'dropout':           0,
    'epochs':            1000,
    'opt':               'adam',
    'opt_scheduler':     'none',
    'opt_restart':       0,
    'weight_decay':      5e-3,
    'lr':                0.0001,
    'device':            device
}

train_loader = DataLoader(
    train_dataset,
    batch_size=args['batch_size'],
    shuffle=True,
    num_workers=4
)

test_loader = DataLoader(
    test_dataset,
    batch_size=args['batch_size'],
    shuffle=True,
    num_workers=4
)

In [ ]:


_, _, META_NN, _ = train(train_loader, test_loader, args)

In [ ]:
save(META_NN, args, 'META_NN')

In [ ]:
META_NN, args = load('META_NN')

In [ ]:
rng = np.random.default_rng(50)
eval_instances, eval_coin_flips = ig.sample_instances(48, 16, 100, rng, **geom_config)

ratios1, log = evaluate_meta_model(
    classify_model=META_NN,
    eval_models=[GNN1, GNN2],
    args=args,
    instances=eval_instances,
    coin_flips=eval_coin_flips,
    batch_size=50,
    rng=rng
)

pp_output(ratios1, log, show_log=True)

In [ ]:
rng = np.random.default_rng(50)
eval_instances, eval_coin_flips = ig.sample_instances(48, 16, 100, rng, **geom_config)

ratios2, log = evaluate_model(
    model=GNN1,
    args=args,
    instances=eval_instances,
    coin_flips=eval_coin_flips,
    batch_size=50,
    rng=rng
)

pp_output(ratios2, log, show_log=True)

In [ ]:
rng = np.random.default_rng(50)
eval_instances, eval_coin_flips = ig.sample_instances(48, 16, 100, rng, **geom_config)

ratios, log = evaluate_model(
    model=GNN2,
    args=args,
    instances=eval_instances,
    coin_flips=eval_coin_flips,
    batch_size=50,
    rng=rng
)

pp_output(ratios, log, show_log=True)

In [ ]:
rng = np.random.default_rng()
eval_instances, eval_coin_flips = ig.sample_instances(32, 32, 10, rng, **ba_config)

X = tc._instances_to_nn_eval(eval_instances).to('cuda')
print(META_NN((X, None)))
